# Density simulation of a path-dependent McKean-Vlasov SDE with linear interaction


> This notebook implements the simulations described in Section 2.1.1 of
  - [1] Bernou, A. and Liu, Y. (2024). Particle method for the numerical simulation of the path-dependent McKean-Vlasov equation

-------------------------------

**Contents**

* [1. Definition of the McKean-Vlasov equation, the particle system and the Euler scheme](#chapter1)

* [2. Application: Simulation with fixed M](#chapter2)

* [3. Application: Simulation with M depending on N](#chapter3)




-------------------------------

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
from joblib import Parallel, delayed
import time
import shutil
from functools import partialmethod
from scipy import special

import pandas as pd
from scipy.stats import norm
from sklearn.linear_model import LinearRegression

from scipy.stats import wasserstein_distance
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

## I. Definition of the McKean-Vlasov equation, the particle system and the Euler scheme <a class="anchor" id="chapter1"></a>


We consider a stochastic process $(X_t)_{t\in[0, T]}$ defined by the following McKean-Vlasov SDE with linear interaction

$$dX_t = - 2 \int_0^t  \int_{\mathbb{R}}(X_t-x)\mu_s(dx) \, ds \, dt+dB_t\quad \text{with}\quad X_0\sim \mathcal{N}(0,1),\hspace{2cm}(1)$$

where for every  $t\in[0,T]$, $\mu_t$ denotes the probability distribution of $X_t$.  This process $(X_t)_{t\in[0, T]}$ is a time-dependent Ornstein-Uhlenbeck process, namely, for every $t\in[0,T], \; \mu_t=\mathcal{N}\big(0, e^{-2t^2}(1 + \int_0^t e^{2 r^2} \, dr) \big)$ (see [1, Proposition 2.1]).

Let $N$ be the number of particles. The $N$-particle system $(X_t^{1}, ..., X_t^{N})_{t\in[0,T]}$ corresponding to (1) is defined as follows :


\begin{equation}
dX_t^{n}=- 2 \int_0^t \frac{1}{N} \sum_{i=1}^{N}\big(X_t^{n}-X_s^{i}\big) ds \, dt+ B_t^{n}.\hspace{2cm}(2),
\end{equation}
where, for all $n \in \{1,\dots,N\}$, $(B^n_t)_{t \ge 0}$ is a standard Brownian motion.

Let $M$ denote the time discretization number for the Euler scheme. Set $h=\frac{T}{M}$ and $t_m=m\cdot h, 0\leq m\leq M$. Combining the particle system (2) and the Euler scheme, we obtain the following discrete particle system (see also [1, Definition 1.4] and [1, equation (1.11)]: for $1 \le n \le N$,

\begin{equation}
X^n_{t_1} = X^n_0 +  \sqrt{h} Z^n_1
\end{equation}
and for $1 \le m \le M-1$,
\begin{align}
X^n_{t_{m+1}} &= X^n_{t_m} + \frac{h^2}{N} \sum_{j=1}^N \Big( \frac{X^j_{t_0}}{2} + \frac{X^j_{t_m}}{2} + \sum_{k=1}^{m-1} X^j_{t_k} \Big) - h \, t_m \, X^n_{t_m} + \sqrt{h} Z^n_{m+1},
\end{align}
where $ \: Z_{m+1}^{n}:=\frac{1}{\sqrt{h}}(B^n_{t_{m+1}}-B^n_{t_{m}})$ are i.i.d random variables having the standard normal distribution $\mathcal{N}(0,1)$.

In the next cell, the function `Euler_one_step` defines the operator of the Euler scheme for one time step.

In [ ]:
def Euler_first_step(X_in, N_in, h):
    nn=np.ones(N_in)/N_in
    t_1 = h
    EXin=np.matmul(nn, X_in[:,0])
    X_in[:, 1] = X_in[:,0] + np.sqrt(h)*np.random.normal(0, 1, N_in)
    return X_in[:,1]

def Euler_one_step(step_in,X_in,N_in, h):
    nn=np.ones(N_in)/N_in
    t_in = step_in*h
    EXin=np.matmul(np.transpose(X_in), nn)
    X_out=(1-2*h*t_in)*X_in[:, step_in]+ (h**2)*(EXin[0] + EXin[step_in]) + 2*(h**2)*(np.sum(EXin[1:step_in-1])) +np.sqrt(h)*np.random.normal(0, 1, N_in)
    return X_out

# II/ Application: Simulation with fixed $M$. <a class="anchor" id="chapter2"></a>
The code of this section is the one used to generate [1, Figure 1].

By default, this section is set with parameter $M=200$.

Modifying the parameters with $M = 10$, $pn = 12$ and $base = 7$ below, this code can also generate [1, Figure 3].

## 1) Simulation of the particle system

In [ ]:
## Parameters
T=1.
M=200  # Time discretization number, adapted in the next sections.
h=T/M  # Time step
m_X0=0
pn = 12
base = 7
N_power=np.linspace(base,base+pn-1,pn)
N_vec=2**N_power.astype(int)
print("M=", M)
print("Vector of the number of particles", N_vec)

First, we compute and save the particle systems at time T=1 with different particle numbers in `N_vec`. For each given particle number, we implement 30 identical and independent simulations.

In [ ]:
def one_sim(particle_num, repeat, M, h):
  particle_process=np.zeros((particle_num,M+1))
  particle_process[:,0]=np.random.normal(m_X0, 1, particle_num)
  particle_process[:,1]=Euler_first_step(particle_process,particle_num, h)

  for m in range(1,M):
    particle_process[:,m+1]=Euler_one_step(m, particle_process,particle_num,h)
  path = "particle_num_"+str(particle_num)+"repeat"+str(repeat)+"step"+str(M)+".csv"
  np.savetxt(path, particle_process[:,M], delimiter=",")
  ## COMPLETE IN THE SECOND LINE INFORMATION ABOUT THE PATH TO SAVE THE DATA
  source_path = path
  destination_path = '/content/drive/MyDrive/Results_OU_M200/'+path ## UNCOMMENT the previous code to save data
  shutil.move(source_path, destination_path)


In [ ]:
# Parallel simulation to run the Monte-Carlo iterations.

## Please check the destination_path in the cell above before running this cell
NNs = 30

start = time.time()
for i in range(pn):
      print("Step"+str(i)+", M = "+str(M)+", N = "+str(N_vec[i]))
      Parallel(n_jobs = 4)(delayed(one_sim)(N_vec[i], j, M, h) for j in tqdm(range(NNs)))
end = time.time()
print(end-start)


The following cell defines the true density function of $X_T$, which is the law $\mathcal{N}(0, e^{-2} (1 + \int_0^1 e^{2r^2} \, dr))$, see [1, Proposition 2.1]. We note that this last variance is $e^{-2} (1 + \tfrac 12 \sqrt{\frac{\pi}{2}} \hbox{erfi}(\sqrt{2}))$ which is approximately equal to $0.45532932051932784$.

In [ ]:
# Definition of the true density function

pi = 3.14159265359
var_approx = np.exp(-2)*(1 + 0.5 * np.sqrt(pi/2)*special.erfi(np.sqrt(2)) )

## 2) Estimation of the Wasserstein distance

In the next cells, we compute an empirical Wasserstein distance, using
$$ \mathcal{W}_2(\mu,\nu)^2 = \int_0^1 \Big|F_\mu^{-1}(q) - F_\nu^{-1}(q) \Big|^2 \, d q. $$
We start by computing the vector of theoretical quantiles for the target distribution in the next cell. Since we cannot compute the full integral with quantiles over $[0,1]$  as this gives infinite values, we use a truncation parameter epsilon.

In [ ]:
def estim_Wp(a,b, epsilon,p): ## takes two vectors of values and compute the estimated integral.
    n = len(a) # Since epsilon=precision_integral below, 1/n = epsilon.
    res = ((1-2*epsilon)/n)*(((a[0] - b[0])**p + (a[-1]-b[-1])**p)/2 + np.sum((a[1:-1] - b[1:-1])**p))
    return(res)

In [ ]:
epsilon = 10**(-6) ##
p = 2
NNs = 30
precision_integral = 10**6 ## This parameter is used for the integral discretization
quantile_vector = np.arange(0.01,1,1/precision_integral)
quantile_theoretical = norm.ppf(quantile_vector, 0, np.sqrt(var_approx))
error_basic = np.zeros((pn, NNs))
max_error = np.zeros((pn, NNs))
error_final = np.zeros(pn)
for i in tqdm(range(pn)):   ## particle number
    for nns in range(NNs):  ## Monte-Carlo repetition
        ## CHANGE THE SECOND LINE below to access data stored somewhere else.
        path = "particle_num_"+str(N_vec[i])+"repeat"+str(nns)+"step"+str(M)+".csv"
        density_particle= np.array(pd.read_csv("/content/drive/MyDrive/Results_OU_M200/"+path, sep=',',header=None))

        ## Estimation of the Wasserstein distance with respect to the true distribution.
        quantile_method = np.quantile(density_particle, quantile_vector)
        error_basic[i, nns] = estim_Wp(quantile_method, quantile_theoretical, epsilon, p)

    ### Computing the finate global error
    error_final[i] = np.mean(error_basic[i,:]**p)








In [ ]:
path = "error_final_M200.csv"
np.savetxt(path, error_final, delimiter=",")

In [ ]:
from google.colab import files
plt.figure(figsize=(9,7))
N_vec_bis = N_vec

# Main plot with larger markers and thicker line
plt.scatter(np.log2(N_vec_bis), np.log2(error_final),
           color="blue", marker="o", s=80, zorder=5,  edgecolors='blue', linewidth=1)
plt.plot(np.log2(N_vec_bis), np.log2(error_final),
         "b-", linewidth=2.5, alpha=0.7)

# Optional: Add theoretical slope reference line
# If you expect a specific convergence rate, e.g., -1/2:
x_theory = np.log2(N_vec_bis)
y_theory = np.log2(error_final[0]) - 0.5 * (x_theory - x_theory[0])
#plt.plot(x_theory, y_theory, '--', color='blue', linewidth=2,
#         alpha=0.8, label="Theoretical slope: -1/4")

# Improve axes and labels
plt.xlabel("$\\log_2(N)$", fontsize=18, fontweight='bold')
plt.ylabel("$\\log_2(\\text{Error})$", fontsize=18, fontweight='bold')
plt.title("Squared error for the estimation of the W_2 distance, $M = 200$",
          fontsize=20, pad=20)

# Add grid for better readability
#plt.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
#plt.grid(True, alpha=0.2, linestyle=':', linewidth=0.5, which='minor')

# Improve tick labels
plt.tick_params(axis='both', which='major', labelsize=12)
plt.tick_params(axis='both', which='minor', labelsize=10)

# Add legend
# plt.legend(fontsize=12, loc='best', framealpha=0.9)

# Tight layout to prevent label cutoff
plt.tight_layout()

# Optional: Add text box with slope information
#slope = np.polyfit(np.log2(N_vec_bis), np.log2(error_final), 1)[0]
#plt.text(0.05, 0.95, f'Observed slope: {slope:.3f}',
#         transform=plt.gca().transAxes, fontsize=12,
 #        bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.8),
  #       verticalalignment='top')

# Save as EPS
#plt.savefig('/content/drive/MyDrive/Results_OU_M200/OU_M200_bis.eps', format='eps', dpi=300, bbox_inches='tight')
#plt.show()
#files.download('/content/drive/MyDrive/Results_OU_M200/OU_M200_bis.eps')

The least-square estimate of the slope of the above curve is the following.

In [ ]:
X=np.log2(N_vec).reshape(-1,1)
y=np.log2(np.sqrt(error_final))
reg_Gaussian = LinearRegression().fit(X, y)
print ("The estimate of the slope is", reg_Gaussian.coef_)
print ("The estimate of the intercept is", reg_Gaussian.intercept_)


# III/ Simulation with $M = N^\alpha$. <a class="anchor" id="chapter3"></a>

## 1) Simulation

The code of this section is used to generate [1, Figure 2].

In [ ]:
# Parameters

T=1.
m_X0=0  # Mean of the distribution of X_0
pn=10
base = 9
alpha = .26
N_power=np.linspace(base,base+pn-1,pn)
print (N_power)
N_vec=2**N_power.astype(int) # Number of particles, from 2^7 to 2^15
M_vec = np.ceil(N_vec**(alpha))
print(N_vec)
print(M_vec)

In [ ]:
## Check the destination path in the function one_sim below before executing this cell

NNs = 30

start = time.time()
for i in range(pn):
      M = M_vec[i].astype(int)
      h = T/M
      print("Step "+str(i)+", M = "+str(M)+", N = "+str(N_vec[i]))
      Parallel(n_jobs = 4)(delayed(one_sim)(N_vec[i], j, M, h) for j in tqdm(range(NNs)))
end = time.time()
print(end-start)


## 2) Estimation of the Wasserstein distance

In [ ]:
epsilon = 10**(-6) ## can't compute int_0^1 with quantiles as this gives infinite values
# epsilon is the truncation value
p = 2
NNs = 30
precision_integral = 10**6
quantile_vector = np.arange(0.01,1,1/precision_integral)
quantile_theoretical = norm.ppf(quantile_vector, 0, np.sqrt(var_approx))
error_basic = np.zeros((pn, NNs))
max_error = np.zeros((pn, NNs))
error_final = np.zeros(pn)
for i in tqdm(range(pn)):   ## particle number
    for nns in range(NNs):  ## Monte-Carlo repetition
        M = int(M_vec[i])
        h = T/M
        #print("N = "+str(N_vec[i])+", M = "+str(M))
        ## Change the second line below to access data stored somewhere else.
        path = "particle_num_"+str(N_vec[i])+"repeat"+str(nns)+"step"+str(M)+".csv"
        density_particle= np.array(pd.read_csv("/content/drive/MyDrive/Results_OU_MN26/"+path, sep=',',header=None))
        ## We have NNs simulations with 2^7 particles for instance. Here we upload the last iteration (result at time T).
        ## Let us estimate the Wasserstein distance with respect to the true distribution.
        quantile_method = np.quantile(density_particle, quantile_vector)
        error_basic[i, nns] = estim_Wp(quantile_method, quantile_theoretical, epsilon, p)

    ### Computing the finate global error
    error_final[i] = np.mean(error_basic[i,:]**p)

In [ ]:
path = "/content/drive/MyDrive/Results_OU_MN26/error_final.csv"
np.savetxt(path, error_final, delimiter=",")

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(np.log2(N_vec),np.log2(np.sqrt(error_final)), color = "blue", marker = "o")

plt.xlabel('log2(N)', fontsize=14) # Adjust fontsize as needed
plt.ylabel('log2(Error)', fontsize=14) # Adjust fontsize as needed
plt.title(r'Squared Error for the estimation of the Wasserstein distance, $M = N^{0.26}$', fontsize=16) # Adjust fontsize as needed
plt.grid(False)

# Increase tick label size
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.savefig('OUMN26.eps', format='eps', bbox_inches='tight')
files.download('OUMN26.eps')

plt.show()

In [ ]:
X=np.log2(N_vec).reshape(-1,1)
y=np.log2(np.sqrt(error_final))
reg_Gaussian = LinearRegression().fit(X, y)
print ("The estimate of the slope is", reg_Gaussian.coef_)
print ("The estimate of the intercept is", reg_Gaussian.intercept_)